### Docker Command Output: `docker exec -it redpanda-1 rpk version`


Version:     v24.2.18
Git ref:     f9a22d4430
Build date:  2025-02-14T12:52:55Z
OS/Arch:     linux/amd64
Go version:  go1.23.1

Redpanda Cluster
  node-1  v24.2.18 - f9a22d443087b824803638623d6b7492ec8221f9

### Docker Command Output: `docker exec -it redpanda-1 rpk topic create green-trips`


TOPIC        STATUS
green-trips  OK

In [4]:
import kafka
kafka.__version__

'2.1.1'

In [1]:
import json

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [3]:
import pandas as pd
import json
from kafka import KafkaProducer
from time import time

# Define Kafka topic and server
topic_name = "green-trips"
server = "localhost:9092"

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Read CSV file
file_path = "green_tripdata_2019-10.csv"
df = pd.read_csv(file_path, usecols=[
    'lpep_pickup_datetime', 'lpep_dropoff_datetime', 
    'PULocationID', 'DOLocationID', 
    'passenger_count', 'trip_distance', 'tip_amount'
])

# Convert timestamp columns to string (JSON doesn't support Timestamp types)
df['lpep_pickup_datetime'] = df['lpep_pickup_datetime'].astype(str)
df['lpep_dropoff_datetime'] = df['lpep_dropoff_datetime'].astype(str)

# Start measuring time
t0 = time()

# Send each row as a Kafka message
for _, row in df.iterrows():
    message = row.to_dict()
    producer.send(topic_name, value=message)

# Flush producer to ensure all messages are sent
producer.flush()

# Stop measuring time
t1 = time()
took = t1 - t0

# Output total time taken
print(f"Time taken to send dataset: {took:.2f} seconds")


Time taken to send dataset: 214.62 seconds
